In [41]:
#coding:utf-8
import pandas as pd
import time
from scipy.sparse import csr_matrix
from scipy.sparse import dok_matrix
from sklearn.metrics import log_loss
import numpy as np
# instance_id 样本编号
# item_id 广告商品编号
# item_category_list 广告商品的的类目列表 分割; item_property_list_0 item_property_list_1 item_property_list_2
# item_property_list 广告商品的属性列表 分割 1 2 3
# item_brand_id 广告商品的品牌编号
# item_city_id 广告商品的城市编号
# item_price_level 广告商品的价格等级
# item_sales_level 广告商品的销量等级
# item_collected_level 广告商品被收藏次数的等级
# item_pv_level 广告商品被展示次数的等级
# user_id 用户的编号
# 'user_gender_id', 用户的预测性别编号
# 'user_age_level', 用户的预测年龄等级
# 'user_occupation_id', 用户的预测职业编号
# 'user_star_level' 用户的星级编号
# context_id 上下文信息的编号
#  context_timestamp 广告商品的展示时间
# context_page_id 广告商品的展示页面编号
# predict_category_property
def time2cov(time_):
    '''
    时间是根据天数推移，所以日期为脱敏，但是时间本身不脱敏
    :param time_: 
    :return: 
    '''
    return time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time_))

def pre_process(data):
    '''
    :param data: 
    :return: 
    '''

    print('预处理')
#     print('item_category_list_ing')
#     for i in range(3):
#         data['category_%d'%(i)] = data['item_category_list'].apply(
#             lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
#         )
#     del data['item_category_list']

#     print('item_property_list_ing')
#     for i in range(3):
#         data['property_%d'%(i)] = data['item_property_list'].apply(
#             lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
#         )
#     del data['item_property_list']

    print('context_timestamp_ing')
    data['context_timestamp'] = data['context_timestamp'].apply(time2cov)

    print('time')
    data['context_timestamp_tmp'] = pd.to_datetime(data['context_timestamp'])
#     data['week'] = data['context_timestamp_tmp'].dt.weekday
    data['hour'] = data['context_timestamp_tmp'].dt.hour
    data['minute'] = data['context_timestamp_tmp'].dt.minute
    data['halfhour'] = (data['hour']+1)*2+(data['minute']/30).astype('int')
    del data['context_timestamp_tmp']
    del data['hour']
    del data['minute']
    del data['item_city_id']
#     print('predict_category_property_ing_0')
#     for i in range(3):
#         data['predict_category_%d'%(i)] = data['predict_category_property'].apply(
#             lambda x:str(x.split(";")[i]).split(":")[0] if len(x.split(";")) > i else " "
#         )
#     for i in range(3):
#         temp = data[['user_occupation_id','property_%d'%(i),'instance_id']].groupby(['user_occupation_id','property_%d'%(i)]).aggregate('count')
#         temp1 = temp.reset_index()
#         temp2 = temp.groupby('user_occupation_id').sum().reset_index()
#         temp3 = pd.merge(temp1,temp2,on=['user_occupation_id'],how='left')
#         temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
#         temp3['property_%dpv'%(i)] = temp4
#         temp3.drop('instance_id_x', axis=1, inplace=True)
#         temp3.drop('instance_id_y', axis=1, inplace=True)
#         data = pd.merge(data,temp3,on=['user_occupation_id','property_%d'%(i)],how='left')
#     for i in range(1,3):
#         temp = data[['user_occupation_id','category_%d'%(i),'instance_id']].groupby(['user_occupation_id','category_%d'%(i)]).aggregate('count')
#         temp1 = temp.reset_index()
#         temp2 = temp.groupby('user_occupation_id').sum().reset_index()
#         temp3 = pd.merge(temp1,temp2,on=['user_occupation_id'],how='left')
#         temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
#         temp3['category_%dpv'%(i)] = temp4
#         temp3.drop('instance_id_x', axis=1, inplace=True)
#         temp3.drop('instance_id_y', axis=1, inplace=True)
#         data = pd.merge(data,temp3,on=['user_occupation_id','category_%d'%(i)],how='left')
#     for i in range(3):
#         temp = data[['user_gender_id','property_%d'%(i),'instance_id']].groupby(['user_gender_id','property_%d'%(i)]).aggregate('count')
#         temp1 = temp.reset_index()
#         temp2 = temp.groupby('user_gender_id').sum().reset_index()
#         temp3 = pd.merge(temp1,temp2,on=['user_gender_id'],how='left')
#         temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
#         temp3['property_%dpv'%(i)] = temp4
#         temp3.drop('instance_id_x', axis=1, inplace=True)
#         temp3.drop('instance_id_y', axis=1, inplace=True)
#         data = pd.merge(data,temp3,on=['user_gender_id','property_%d'%(i)],how='left')
#     for i in range(1,3):
#         temp = data[['user_gender_id','category_%d'%(i),'instance_id']].groupby(['user_gender_id','category_%d'%(i)]).aggregate('count')
#         temp1 = temp.reset_index()
#         temp2 = temp.groupby('user_gender_id').sum().reset_index()
#         temp3 = pd.merge(temp1,temp2,on=['user_gender_id'],how='left')
#         temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
#         temp3['category_%dpv'%(i)] = temp4
#         temp3.drop('instance_id_x', axis=1, inplace=True)
#         temp3.drop('instance_id_y', axis=1, inplace=True)
#         data = pd.merge(data,temp3,on=['user_gender_id','category_%d'%(i)],how='left')
#     temp = data[['user_age_level','item_price_level','instance_id']].groupby(['user_age_level','item_price_level']).aggregate('count')
#     temp1 = temp.reset_index()
#     temp2 = temp.groupby('user_age_level').sum().reset_index()
#     temp3 = pd.merge(temp1,temp2,on=['user_age_level'],how='left')
#     temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
#     temp3['age_pv'] = temp4
#     temp3.drop('instance_id_x', axis=1, inplace=True)
#     temp3.drop('instance_id_y', axis=1, inplace=True)
#     data = pd.merge(data,temp3,on=['user_age_level','item_price_level'],how='left')
#     del(temp3)
#     del(temp2)
#     del(temp1)
#     del(temp4)
#     del(temp)
#     print('predict_category_property_ing_1')
#     for i in range(3):
#         data['predict_property_%d'%(i)] = data['predict_category_property'].apply(
#             lambda x:str(x.split(";")[i]).split(":")[1] if len(x.split(";")) > i else " "
#         )
    
#         for j in range(3):
#             data['predict_property_%d_%d' % (i,j)] = data['predict_property_%d'%(i)].apply(
#                 lambda x: x.split(",")[j] if len(x.split(",")) > j else -1
#             )

    del data['predict_category_property']
    # del data['predict_property_1']
    # del data['predict_property_2']

    return data

In [42]:
print('train')
train = pd.read_csv('train.txt',sep=" ")
data2 = pd.read_csv('1234.csv',index_col=False) 
datac = pd.read_csv('clickpv1.csv',index_col=False) 
train.insert(1, 'item_category_list_num', data2[['item_category_list']])
train.insert(2, 'item_property_list_num', data2[['item_property_list']])
train.insert(1, 'clickpv', datac[[ 'clickpv']])
train = pre_process(train)

all_data = train.copy()


print('all_shape',train.shape)
print(train['context_timestamp'].max())
val = train[train['context_timestamp']>'2018-09-22 23:59:59']

# 空去 22 号

train = train[train['context_timestamp']<='2018-09-22 23:59:59']

train = train[train['context_timestamp']>'2018-09-19 23:59:59']
print(train.shape)
print(val.shape)



train
预处理
context_timestamp_ing
time
all_shape (478138, 29)
2018-09-24 23:59:47
(207904, 29)
(121035, 29)


In [43]:
# train['halfhour']

In [44]:
train.columns

Index(['instance_id', 'clickpv', 'item_category_list_num',
       'item_property_list_num', 'item_id', 'item_category_list',
       'item_property_list', 'item_brand_id', 'item_price_level',
       'item_sales_level', 'item_collected_level', 'item_pv_level', 'user_id',
       'user_gender_id', 'user_age_level', 'user_occupation_id',
       'user_star_level', 'context_id', 'context_timestamp', 'context_page_id',
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'is_trade', 'halfhour'],
      dtype='object')

In [45]:
print('test')
test_a = pd.read_csv('test.txt',sep=" ")
data3 = pd.read_csv('456.csv',index_col=False)
data2c = pd.read_csv('test_clickpv1.csv',index_col=False) 
# data2.pop('item_category_list')
test_a.insert(1, 'item_category_list_num', data3[['item_category_list']])
# data2.pop('item_property_list')
test_a.insert(2, 'item_property_list_num', data3[['item_property_list']])
test_a.insert(1, 'clickpv', data2c[[ 'clickpv']])
print(test_a.shape)
test_a = pre_process(test_a)

test
(18371, 29)
预处理
context_timestamp_ing
time


In [46]:
test_a

,instance_id,clickpv,item_category_list_num,item_property_list_num,item_id,item_category_list,item_property_list,item_brand_id,item_price_level,item_sales_level,...,context_timestamp,context_page_id,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,halfhour
0,2475218615076601065,0.01,2,1,2275895163219263378,7908382889764677758;8277336076276184272,2636395404473730413;7515802706813700848;302719...,7838285046767229711,7,6,...,2018-09-25 22:28:20,4005,5275548996133547595,13,0.985244,5011,0.964023,0.966550,0.955378,46
1,398316874173557226,0.01,2,0,7096238490711246967,7908382889764677758;5755694407684602296,5131280576272319091;2636395404473730413;734498...,8126195666233054089,6,9,...,2018-09-25 16:07:23,4001,329470548383752413,14,0.989068,5012,0.963794,0.964225,0.959134,34
2,6586402638209028583,0.01,2,2,7096238490711246967,7908382889764677758;5755694407684602296,5131280576272319091;2636395404473730413;734498...,8126195666233054089,6,9,...,2018-09-25 14:50:22,4011,329470548383752413,14,0.989068,5012,0.963794,0.964225,0.959134,31
3,1040996105851528465,0.01,2,1,7096238490711246967,7908382889764677758;5755694407684602296,5131280576272319091;2636395404473730413;734498...,8126195666233054089,6,9,...,2018-09-25 15:35:15,4001,329470548383752413,14,0.989068,5012,0.963794,0.964225,0.959134,33
4,6316278569655873454,0.01,2,4,7096238490711246967,7908382889764677758;5755694407684602296,5131280576272319091;2636395404473730413;734498...,8126195666233054089,6,9,...,2018-09-25 15:04:34,4001,329470548383752413,14,0.989068,5012,0.963794,0.964225,0.959134,32
5,868158305045921978,0.01,2,0,7096238490711246967,7908382889764677758;5755694407684602296,5131280576272319091;2636395404473730413;734498...,8126195666233054089,6,9,...,2018-09-25 16:02:57,4001,329470548383752413,14,0.989068,5012,0.963794,0.964225,0.959134,34
6,5713520501786699854,0.01,2,0,1171140458228333440,7908382889764677758;4879721024980945592,5131280576272319091;2636395404473730413;316326...,1547983184598012595,5,6,...,2018-09-25 11:35:09,4001,7599689351860250922,16,1.000000,5013,0.976897,0.977126,0.984791,25
7,932945015407923184,0.01,2,1,7020520887593189887,7908382889764677758;2011981573061447208,5131280576272319091;1013419034389836991;519513...,8324857885829459863,7,10,...,2018-09-25 13:11:09,4001,7565455264321824526,15,1.000000,5012,0.979252,0.980538,0.985626,28
8,1919197847086752313,0.01,2,0,8143258757857432437,7908382889764677758;2011981573061447208,2636395404473730413;4621934203383159480;124376...,8324857885829459863,8,10,...,2018-09-25 11:28:03,4001,7565455264321824526,15,1.000000,5012,0.979252,0.980538,0.985626,24
9,304887065966615346,0.01,2,1,8143258757857432437,7908382889764677758;2011981573061447208,2636395404473730413;4621934203383159480;124376...,8324857885829459863,8,10,...,2018-09-25 14:09:08,4001,7565455264321824526,15,1.000000,5012,0.979252,0.980538,0.985626,30


In [47]:
# 这里是增加的内容
import datetime
def get_count_feat(all_data,data,long=3):
    end_time = data['context_timestamp'].min()
    begin_time = pd.to_datetime(end_time) - datetime.timedelta(days=long)
    all_data['context_timestamp'] = pd.to_datetime(all_data['context_timestamp'])
    all_data = all_data[
        (all_data['context_timestamp']<end_time)&(all_data['context_timestamp']>=begin_time)
                    ]
    print(end_time)
    print(begin_time)
    print(all_data['context_timestamp'].max()-all_data['context_timestamp'].min())
    item_count = all_data.groupby(['item_id'],as_index=False).size().reset_index()
    item_count.rename(columns={0:'item_count'},inplace=True)

    user_count = all_data.groupby(['user_id'], as_index=False).size().reset_index()
    user_count.rename(columns={0: 'user_count'}, inplace=True)
    return user_count,item_count


In [48]:

train_user_count,train_item_count = get_count_feat(all_data,train,2)

test_user_count,test_item_count = get_count_feat(all_data,test_a,2)

val_user_count,val_item_count = get_count_feat(all_data,val,2)

train = pd.merge(train,train_user_count,on=['user_id'],how='left')
train = pd.merge(train,train_item_count,on=['item_id'],how='left')
train = train.fillna(-1)
val = pd.merge(val,val_user_count,on=['user_id'],how='left')
val = pd.merge(val,val_item_count,on=['item_id'],how='left')
val = val.fillna(-1)
test_a = pd.merge(test_a,test_user_count,on=['user_id'],how='left')
test_a = pd.merge(test_a,test_item_count,on=['item_id'],how='left')
test_a = test_a.fillna(-1)
y_train = train.pop('is_trade')
train_index = train.pop('instance_id')

y_val = val.pop('is_trade')
val_index = val.pop('instance_id')
test_index = test_a.pop('instance_id')



2018-09-20 00:00:01
2018-09-18 00:00:01
1 days 23:59:54
2018-09-25 00:00:02
2018-09-23 00:00:02
1 days 23:59:44
2018-09-23 00:00:03
2018-09-21 00:00:03
1 days 23:59:43


In [49]:
print(test_a.shape)
del train['context_timestamp']
del val['context_timestamp']
del test_a['context_timestamp']
del all_data



(18371, 29)


In [50]:
print('baseline ing ... ...')
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from scipy import sparse
from sklearn.linear_model import LogisticRegression
print(test_a.columns)

enc = OneHotEncoder()
lb = LabelEncoder()
feat_set = list(test_a.columns)
for i,feat in enumerate(feat_set):
    if feat == 'shop_review_positive_rate' or feat == 'shop_score_service' or feat == 'shop_score_delivery' or feat == 'shop_score_description':
#         print("aaaaaaaaaaa")
#         print(X_train.shape[0])
#         print("aaaaaaaaaaa")
        temp1 = csr_matrix( (np.array(np.array(train)[:,i],dtype = 'float64').ravel(), (np.arange(train.shape[0]).ravel(), np.full([train.shape[0],1], 0).ravel()) ), shape=(X_train.shape[0], 1),dtype='double')
        temp2 = csr_matrix( (np.array(np.array(test_a)[:,i],dtype = 'float64').ravel(), (np.arange(test_a.shape[0]).ravel(), np.full([test_a.shape[0],1], 0).ravel()) ), shape=(X_test.shape[0], 1),dtype='double')
        temp3 = csr_matrix( (np.array(np.array(val)[:,i],dtype = 'float64').ravel(), (np.arange(val.shape[0]).ravel(), np.full([val.shape[0],1], 0).ravel()) ), shape=(val.shape[0], 1),dtype='double')
        X_train = sparse.hstack((X_train, temp1))
        X_test = sparse.hstack((X_test, temp2))
        X_val = sparse.hstack((X_val, temp3))
        continue
    if feat == 'context_id' or feat == 'item_city_id':
        continue
    tmp = lb.fit_transform((list(train[feat])+list(val[feat])+list(test_a[feat])))
    enc.fit(tmp.reshape(-1,1))
    x_train = enc.transform(lb.transform(train[feat]).reshape(-1, 1))
    x_test = enc.transform(lb.transform(test_a[feat]).reshape(-1, 1))
    x_val = enc.transform(lb.transform(val[feat]).reshape(-1, 1))
    if i == 0:
        X_train, X_test,X_val = x_train, x_test,x_val
    else:
        X_train, X_test,X_val = sparse.hstack((X_train, x_train)), sparse.hstack((X_test, x_test)),sparse.hstack((X_val, x_val))
X_train, X_test,X_val = sparse.hstack((X_train, x_train)), sparse.hstack((X_test, x_test)),sparse.hstack((X_val, x_val))

baseline ing ... ...
Index(['clickpv', 'item_category_list_num', 'item_property_list_num',
       'item_id', 'item_category_list', 'item_property_list', 'item_brand_id',
       'item_price_level', 'item_sales_level', 'item_collected_level',
       'item_pv_level', 'user_id', 'user_gender_id', 'user_age_level',
       'user_occupation_id', 'user_star_level', 'context_id',
       'context_page_id', 'shop_id', 'shop_review_num_level',
       'shop_review_positive_rate', 'shop_star_level', 'shop_score_service',
       'shop_score_delivery', 'shop_score_description', 'halfhour',
       'user_count', 'item_count'],
      dtype='object')


In [51]:
# train.shape[0]#(207904, 574259)
# np.array(train)[:,24].ravel()

In [52]:
# row = [2,1,3,0]
# col = [0,0,0,0]
# data =[0.9757697121401752,2,3,3]
# c = sparse.coo_matrix((data,(row,col)),shape=(4,1))
# c.toarray()

In [53]:
# np.array(np.array(train)[:,24],dtype = 'double').ravel()

In [54]:
# train.columns20,22,23,24
# np.array(train)[:,24]
# # train.columns
# temp = csr_matrix( (np.array(np.array(train)[:,24],dtype = 'float64').ravel(), (np.arange(train.shape[0]).ravel(), np.full([train.shape[0],1], 0).ravel()) ), shape=(train.shape[0], 1),dtype='double')
# sparse.hstack((X_train, x_train))

In [55]:
lr = LogisticRegression()


lr.fit(X_train, y_train)
proba_val = lr.predict_proba(X_val)[:,1]
proba_sub = lr.predict_proba(X_test)[:,1]

In [56]:
print(log_loss(y_train,lr.predict_proba(X_train)[:,1]))
print(log_loss(y_val,proba_val))#0.0855354191184

0.0657713054695
0.0855354191184


In [57]:
# lgb.LGBMClassifier?

In [58]:
%%time
import lightgbm as lgb

gbm = lgb.LGBMClassifier(objective='binary',
                        num_leaves=64,
                        learning_rate=0.01,
                        n_estimators=2000,
                        colsample_bytree = 0.65,
                        subsample = 0.75,random_state =666,
                        gpu_platform_id = 0, gpu_device_id = 0
                        ,reg_alpha = 2,num_iterations=2000
                       )
gbm.fit(X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='binary_logloss',
        early_stopping_rounds=150)

D:\anaconda\lib\site-packages\lightgbm\engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's binary_logloss: 0.683895
Training until validation scores don't improve for 150 rounds.
[2]	valid_0's binary_logloss: 0.674827
[3]	valid_0's binary_logloss: 0.665933
[4]	valid_0's binary_logloss: 0.657211
[5]	valid_0's binary_logloss: 0.648662
[6]	valid_0's binary_logloss: 0.640273
[7]	valid_0's binary_logloss: 0.632046
[8]	valid_0's binary_logloss: 0.623971
[9]	valid_0's binary_logloss: 0.616049
[10]	valid_0's binary_logloss: 0.60827
[11]	valid_0's binary_logloss: 0.60063
[12]	valid_0's binary_logloss: 0.593139
[13]	valid_0's binary_logloss: 0.585778
[14]	valid_0's binary_logloss: 0.578554
[15]	valid_0's binary_logloss: 0.571451
[16]	valid_0's binary_logloss: 0.564481
[17]	valid_0's binary_logloss: 0.557629
[18]	valid_0's binary_logloss: 0.550895
[19]	valid_0's binary_logloss: 0.544281
[20]	valid_0's binary_logloss: 0.537775
[21]	valid_0's binary_logloss: 0.531383
[22]	valid_0's binary_logloss: 0.525102
[23]	valid_0's binary_logloss: 0.518926
[24]	valid_0's binary_loglos

[204]	valid_0's binary_logloss: 0.125286
[205]	valid_0's binary_logloss: 0.124742
[206]	valid_0's binary_logloss: 0.12421
[207]	valid_0's binary_logloss: 0.123683
[208]	valid_0's binary_logloss: 0.12316
[209]	valid_0's binary_logloss: 0.122645
[210]	valid_0's binary_logloss: 0.122132
[211]	valid_0's binary_logloss: 0.121628
[212]	valid_0's binary_logloss: 0.121131
[213]	valid_0's binary_logloss: 0.120639
[214]	valid_0's binary_logloss: 0.120152
[215]	valid_0's binary_logloss: 0.119671
[216]	valid_0's binary_logloss: 0.119198
[217]	valid_0's binary_logloss: 0.118733
[218]	valid_0's binary_logloss: 0.11827
[219]	valid_0's binary_logloss: 0.117816
[220]	valid_0's binary_logloss: 0.117364
[221]	valid_0's binary_logloss: 0.116919
[222]	valid_0's binary_logloss: 0.11648
[223]	valid_0's binary_logloss: 0.116049
[224]	valid_0's binary_logloss: 0.115624
[225]	valid_0's binary_logloss: 0.115201
[226]	valid_0's binary_logloss: 0.114785
[227]	valid_0's binary_logloss: 0.114375
[228]	valid_0's bina

[402]	valid_0's binary_logloss: 0.0861963
[403]	valid_0's binary_logloss: 0.0861575
[404]	valid_0's binary_logloss: 0.086121
[405]	valid_0's binary_logloss: 0.0860836
[406]	valid_0's binary_logloss: 0.0860475
[407]	valid_0's binary_logloss: 0.0860116
[408]	valid_0's binary_logloss: 0.0859775
[409]	valid_0's binary_logloss: 0.085944
[410]	valid_0's binary_logloss: 0.0859085
[411]	valid_0's binary_logloss: 0.0858755
[412]	valid_0's binary_logloss: 0.0858432
[413]	valid_0's binary_logloss: 0.0858102
[414]	valid_0's binary_logloss: 0.085778
[415]	valid_0's binary_logloss: 0.0857474
[416]	valid_0's binary_logloss: 0.0857137
[417]	valid_0's binary_logloss: 0.0856831
[418]	valid_0's binary_logloss: 0.0856514
[419]	valid_0's binary_logloss: 0.08562
[420]	valid_0's binary_logloss: 0.0855914
[421]	valid_0's binary_logloss: 0.0855606
[422]	valid_0's binary_logloss: 0.0855324
[423]	valid_0's binary_logloss: 0.0855034
[424]	valid_0's binary_logloss: 0.0854738
[425]	valid_0's binary_logloss: 0.08544

[601]	valid_0's binary_logloss: 0.0835566
[602]	valid_0's binary_logloss: 0.0835511
[603]	valid_0's binary_logloss: 0.0835476
[604]	valid_0's binary_logloss: 0.0835457
[605]	valid_0's binary_logloss: 0.0835418
[606]	valid_0's binary_logloss: 0.0835367
[607]	valid_0's binary_logloss: 0.0835348
[608]	valid_0's binary_logloss: 0.0835302
[609]	valid_0's binary_logloss: 0.0835258
[610]	valid_0's binary_logloss: 0.0835226
[611]	valid_0's binary_logloss: 0.0835195
[612]	valid_0's binary_logloss: 0.0835154
[613]	valid_0's binary_logloss: 0.0835146
[614]	valid_0's binary_logloss: 0.0835136
[615]	valid_0's binary_logloss: 0.0835116
[616]	valid_0's binary_logloss: 0.083508
[617]	valid_0's binary_logloss: 0.0835053
[618]	valid_0's binary_logloss: 0.0835029
[619]	valid_0's binary_logloss: 0.0835
[620]	valid_0's binary_logloss: 0.0834978
[621]	valid_0's binary_logloss: 0.0834926
[622]	valid_0's binary_logloss: 0.0834887
[623]	valid_0's binary_logloss: 0.0834884
[624]	valid_0's binary_logloss: 0.0834

[797]	valid_0's binary_logloss: 0.0832776
[798]	valid_0's binary_logloss: 0.0832784
[799]	valid_0's binary_logloss: 0.0832785
[800]	valid_0's binary_logloss: 0.0832784
[801]	valid_0's binary_logloss: 0.0832788
[802]	valid_0's binary_logloss: 0.083278
[803]	valid_0's binary_logloss: 0.0832772
[804]	valid_0's binary_logloss: 0.0832768
[805]	valid_0's binary_logloss: 0.083277
[806]	valid_0's binary_logloss: 0.0832768
[807]	valid_0's binary_logloss: 0.0832761
[808]	valid_0's binary_logloss: 0.0832759
[809]	valid_0's binary_logloss: 0.0832744
[810]	valid_0's binary_logloss: 0.0832755
[811]	valid_0's binary_logloss: 0.0832748
[812]	valid_0's binary_logloss: 0.0832734
[813]	valid_0's binary_logloss: 0.0832751
[814]	valid_0's binary_logloss: 0.0832726
[815]	valid_0's binary_logloss: 0.0832713
[816]	valid_0's binary_logloss: 0.0832721
[817]	valid_0's binary_logloss: 0.0832726
[818]	valid_0's binary_logloss: 0.0832732
[819]	valid_0's binary_logloss: 0.0832726
[820]	valid_0's binary_logloss: 0.08

[996]	valid_0's binary_logloss: 0.0832389
[997]	valid_0's binary_logloss: 0.0832375
[998]	valid_0's binary_logloss: 0.0832362
[999]	valid_0's binary_logloss: 0.0832359
[1000]	valid_0's binary_logloss: 0.0832346
[1001]	valid_0's binary_logloss: 0.0832345
[1002]	valid_0's binary_logloss: 0.0832352
[1003]	valid_0's binary_logloss: 0.0832369
[1004]	valid_0's binary_logloss: 0.0832364
[1005]	valid_0's binary_logloss: 0.0832364
[1006]	valid_0's binary_logloss: 0.0832372
[1007]	valid_0's binary_logloss: 0.083237
[1008]	valid_0's binary_logloss: 0.0832367
[1009]	valid_0's binary_logloss: 0.0832368
[1010]	valid_0's binary_logloss: 0.0832387
[1011]	valid_0's binary_logloss: 0.0832376
[1012]	valid_0's binary_logloss: 0.0832399
[1013]	valid_0's binary_logloss: 0.0832426
[1014]	valid_0's binary_logloss: 0.0832428
[1015]	valid_0's binary_logloss: 0.0832426
[1016]	valid_0's binary_logloss: 0.0832421
[1017]	valid_0's binary_logloss: 0.0832407
[1018]	valid_0's binary_logloss: 0.0832405
[1019]	valid_0's

[1190]	valid_0's binary_logloss: 0.0832453
[1191]	valid_0's binary_logloss: 0.0832449
[1192]	valid_0's binary_logloss: 0.0832454
[1193]	valid_0's binary_logloss: 0.0832451
[1194]	valid_0's binary_logloss: 0.083244
[1195]	valid_0's binary_logloss: 0.0832447
[1196]	valid_0's binary_logloss: 0.083244
[1197]	valid_0's binary_logloss: 0.0832434
[1198]	valid_0's binary_logloss: 0.0832447
[1199]	valid_0's binary_logloss: 0.0832439
[1200]	valid_0's binary_logloss: 0.083246
[1201]	valid_0's binary_logloss: 0.0832451
[1202]	valid_0's binary_logloss: 0.0832426
[1203]	valid_0's binary_logloss: 0.0832418
[1204]	valid_0's binary_logloss: 0.0832413
[1205]	valid_0's binary_logloss: 0.0832416
[1206]	valid_0's binary_logloss: 0.0832395
[1207]	valid_0's binary_logloss: 0.0832407
[1208]	valid_0's binary_logloss: 0.0832412
[1209]	valid_0's binary_logloss: 0.0832413
[1210]	valid_0's binary_logloss: 0.0832403
[1211]	valid_0's binary_logloss: 0.0832403
[1212]	valid_0's binary_logloss: 0.0832407
[1213]	valid_0

In [59]:
print('Start predicting...')
# predict
y_pred_1 = gbm.predict_proba(X_val, num_iteration=gbm.best_iteration_ )[:,1]
y_sub_1 = gbm.predict_proba(X_test, num_iteration=gbm.best_iteration_ )[:,1]
print()
print(log_loss(y_train,gbm.predict_proba(X_train)[:,1]))
print(log_loss(y_val,y_pred_1))

print(log_loss(y_val,(y_pred_1 + proba_val)/2))#0.0830389603803

Start predicting...


D:\anaconda\lib\site-packages\lightgbm\basic.py:447: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')



0.0795768003956
0.0832224062191
0.0834249002369


In [325]:
%%time
import xgboost as xgb

model_xgb = xgb.XGBClassifier( n_estimators=500,random_state =666, nthread = -1,n_jobs=-1,max_depth=10,alpha = 4,eta=0.01,min_child_weight=5)
model_xgb.fit(X_train, y_train,eval_metric='logloss')
# param['gpu_id'] = 0
# param['tree_method'] = 'gpu_hist' # or 'gpu_exact'

y_xpred_1 = model_xgb.predict_proba(X_val)[:,1]
y_xsub_1 = model_xgb.predict_proba(X_test)[:,1]


Wall time: 2min 50s


In [326]:
print(log_loss(y_train,model_xgb.predict_proba(X_train)))
print(log_loss(y_val, y_xpred_1))

0.0654611361752
0.085896819987


In [327]:
print(log_loss(y_val,(y_pred_1 + y_xpred_1)/2))

0.0837107706116


In [341]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingClassifier
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import dok_matrix

In [ ]:
# ENet = make_pipeline(ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
# lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
# KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
# GBoost = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.05,
#                                    max_depth=40, max_features='sqrt',
#                                    min_samples_leaf=150, min_samples_split=100,  random_state =5)

In [349]:
# ENet.fit(X_train, y_train)

In [350]:
# epre = ENet.predict(X_val)

In [332]:
log_loss(y_val,epre)

0.085154230118305835

In [347]:
# GBoost.fit(X_train, y_train)

In [ ]:
# gpre = GBoost.predict(X_val)

In [348]:
# log_loss(y_val,gpre)

In [ ]:
import numpy as np
bes = 1
for i in np.arange(0,1,0.01):
    if(log_loss(y_val,(i * y_pred_1 + (1-i) * proba_val)) < bes):
        bes = log_loss(y_val,(y_pred_1 + proba_val)/2)
print(bes)#0.083131236937

In [293]:
xx_analyse = pd.DataFrame()
xx_analyse['ture'] = list(y_val)
xx_analyse['pre'] = list(proba_val)
xx_analyse['pre_1'] = list(y_pred_1)
xx_analyse.to_csv('temp.csv',index=False)


sub = pd.DataFrame()
sub['instance_id'] = list(test_index)
sub['predicted_score'] = list(y_sub_1)
sub.to_csv('20180322.txt',sep=" ",index=False)

In [403]:
sub = pd.DataFrame()
sub['instance_id'] = list(test_index)
sub['predicted_score'] = list(y_sub_1)
sub.to_csv('20180327.txt',sep=" ",index=False)

In [ ]:
(proba_sub + y_sub_1)/2